In [1]:
from fastai2.vision.all import *
import sys
sys.path.append('..')

In [2]:
%load_ext autoreload
%autoreload 2

from modules.trainutils import build_dl, piece_class_parse
from modules.trainutils import my_metrics, stratify_sample
from modules.trainutils import show_cf

In [3]:
from modules.trainutils import subcat_color_acc, subcat_piece_acc

from modules.trainutils import SecondRecorder

In [4]:
test_path = Path('../../../other-chess-data/regulation-pieces-1/crops/')
train_path = Path('../../../rf-chess-data/cropped_v1/')

In [5]:
test_dl = build_dl(test_path, n=None)

In [34]:
len(test_dl.train.items)

60

In [7]:
n = 200
seed=42

train_dl = ImageDataLoaders.from_name_func(
                train_path, 
                # get_image_files(train_path),
                stratify_sample(train_path, n=n, np_seed=seed),
                valid_pct=0.2, 
                seed=42,
                label_func=piece_class_parse, 
                item_tfms=RandomResizedCrop(128, min_scale=0.5),
                batch_tfms=aug_transforms(),
)

In [8]:
learn = cnn_learner(train_dl, resnet18, 
                    metrics=[accuracy, 
                             subcat_color_acc, 
                             subcat_piece_acc,
                            ]
                   )

In [9]:
new_dl = DataLoaders(learn.dls[0], learn.dls[1], test_dl.train)
learn.dls = new_dl

In [10]:
learn.add_cb(SecondRecorder(ds_idsx=2))

In [11]:
learn.fit_one_cycle(2)

epoch,train_loss,valid_loss,accuracy,subcat_color_acc,subcat_piece_acc,error_rate,top_k_accuracy,time
0,3.813400,6.486135,0.131579,0.578947,0.210526,0.868421,0.631579,00:09
1,3.652573,3.676883,0.131579,0.657895,0.236842,0.868421,0.710526,00:10


##### Extract stuff from recorders

In [12]:
learn.cbs

(#4) [TrainEvalCallback,Recorder,ProgressCallback,SecondRecorder]

In [13]:
rec = learn.cbs[1]
sr = learn.cbs[-1]

In [30]:
df_recorder = pd.DataFrame(rec.values, 
                           columns = rec.metric_names[1:-1]
                          )
df_recorder.T

,0,1
train_loss,3.813400,3.652573
valid_loss,6.486135,3.676883
accuracy,0.131579,0.131579
subcat_color_acc,0.578947,0.657895
subcat_piece_acc,0.210526,0.236842
error_rate,0.868421,0.868421
top_k_accuracy,0.631579,0.710526


In [31]:
met_names = [e.name for e in rec.metrics]
met_names

['accuracy',
 'subcat_color_acc',
 'subcat_piece_acc',
 'error_rate',
 'top_k_accuracy']

In [33]:
cols = ['loss']
cols += met_names
cols = ['test_' + col for col in cols]

df_second_recorder = pd.DataFrame(sr.values, columns = cols)

df_second_recorder.T

,0,1
test_loss,6.486135,3.676883
test_accuracy,0.131579,0.131579
test_subcat_color_acc,0.578947,0.657895
test_subcat_piece_acc,0.210526,0.236842
test_error_rate,0.868421,0.868421
test_top_k_accuracy,0.631579,0.710526
